In [ ]:
# Mnist 手写数字图形识别
from __future__ import print_function
import os 
import struct 
import numpy as np 
# 加载mnist
def load_mnist(path,kind='train'):
    labels_path = os.path.join(path,'%s-labels.idx1-ubyte' % kind)